In [19]:
import csv
import ast
import sys
import pandas as pd

csv.field_size_limit(sys.maxsize)

# File paths
first_csv = 'data/final_comments.csv'
second_csv = 'data/top_reddit_posts.csv'
output_csv = 'output_file.csv'

# Function to remove whitespace from a string
def normalize_text(text):
    return ''.join(text.split())  # Remove all whitespace

# Load the first CSV into a pandas DataFrame
df_first = pd.read_csv(first_csv)

# Prepare a dictionary to map normalized comment content to comment ID
comment_content_to_id = {}

# Process the second CSV to extract Comment Content and Comment ID
with open(second_csv, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        comments = ast.literal_eval(row['Comments'])  # Convert string to list of dictionaries
        for comment in comments:
            comment_content = comment.get('Comment Content', '')
            comment_id = comment.get('Comment ID', '').strip()
            if comment_content:
                # Normalize the comment content for comparison
                normalized_content = normalize_text(comment_content)
                comment_content_to_id[normalized_content] = comment_id

# Map comment_content from the first CSV to the Comment ID
df_first['comment_id'] = df_first['comment_content'].map(
    lambda content: comment_content_to_id.get(normalize_text(content), None)
)

# Save the updated DataFrame to a new CSV file
df_first.to_csv(output_csv, index=False)

print(f"Updated CSV file with comment_id added saved to {output_csv}")



Updated CSV file with comment_id added saved to output_file.csv


In [31]:

# Load the CSV files into dataframes
csv1 = pd.read_csv("data/id_added.csv")
csv2 = pd.read_csv("data/comment_sentiment.csv")

# Merge the dataframes using a left join to keep all rows from csv1 and add matches from csv2
merged_df = pd.merge(csv1, csv2[['comment_id', 'RoBERTa_sentiment']], on='comment_id', how='left')

# Reorder columns as desired
final_columns = ['post_id', 'comment_id', 'comment_content', 'comment_score', 
                 'comment_created_utc', 'length_category', 'sentiment', 'RoBERTa_sentiment']
final_df = merged_df[final_columns]

# Save the new dataframe to a CSV file
final_df.to_csv("selected_comments_with_RoBERTa.csv", index=False)

print("Merged file saved as 'merged_output.csv'")


Merged file saved as 'merged_output.csv'
